# Задание 4 - Реализация сверточной нейронной сети на Keras

В этом упражнении мы реализуем:
- функцию для вычисления 2-мерной свертки  
- функцию для вычисления 3-мерной свертки  
- сверточную сеть на Keras и обучим ее до точности 80%

### 1D convolution

$$ c_{i}=\sum \limits _{k=0}^{N_{out}}x_{k}w_{i-k},\;i=0,\ldots ,N_{out} $$

In [ ]:
import numpy as np

x = np.array((0,0,1,1,1,0,0))
w = np.array((0,1,0))
c = np.zeros(5)

for i in range(5):
    c[i] = np.sum(x[i:i+3]*w)
print(c)

### 2D convolution

In [ ]:
x = np.array([
    [0,0,1,1,0,0,0],
    [0,0,1,1,0,0,0],
    [0,1,1,1,0,0,0],
    [0,0,1,1,0,0,0],
    [0,0,1,1,0,0,0],
    [0,0,1,1,0,0,0]
    ])

In [ ]:
w1 = np.array([
    [0,0,0],
    [1,1,1],
    [0,0,0]
])

w2 = np.array([
    [0,1,0],
    [0,1,0],
    [0,1,0]
])

In [ ]:
def conv2D(x, w):
    c = np.zeros((4,5))
    for i in range(4):
        for j in range(5):
            c[i,j] = np.sum(x[i:i+3, j:j+3] * w)
    return c

In [ ]:
conv2D(x, w1)

In [ ]:
conv2D(x, w2)

### Задание 1. 
Перепишите функцию conv2D так, чтобы она работала с 2D матрицами x и w произвольного размера

In [ ]:
def conv2D(x, w):
    # Your code here
    
    return c

In [ ]:
c = conv2D(np.ones((10, 20)), np.ones((3,4)))

assert c.shape == (8, 17)

### Задание 2. 
Напишите функцию conv2D так, чтобы она работала с 3D тензорами x и w с произвольными высотой и шириной и глубиной 3

In [73]:
def conv2D(x, w):
    # Your code here
    
    return c

In [ ]:
c = conv2D(np.ones((10, 20, 3)), np.ones((3,4,3)))

assert c.shape == (8, 17, 1)

# Keras  Model

In [ ]:
%%bash
git clone https://github.com/balezz/modern_dl.git
cd modern_dl
mkdir data
cd data
wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz -O cifar-10-python.tar.gz > /dev/null
tar -xzvf cifar-10-python.tar.gz > /dev/null

In [ ]:
%cd modern_dl

In [ ]:
import random
import numpy as np
from lib.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots

In [ ]:
# Путь к папке с данными
cifar10_dir = 'data/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# Нормализуем значения яркости пикселей 
mean_image = np.mean(X_train, axis=0)

# Вычтем средние значения яркости
X_train -= mean_image
X_test -= mean_image

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers

class MyModel(tf.keras.Model):

    def __init__(self, input_shape):
        super().__init__()
        self.conv = layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), 
                                   activation='tanh', input_shape=input_shape, padding="same")
        self.pool = layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(84, activation='tanh')
        self.dense2 = layers.Dense(10, activation='softmax')

    def call(self, inputs, training=False):
        x = self.conv(inputs)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        
        return x


In [72]:
# Create ConvNet instance
IMAGE_SHAPE = (32, 32, 3)

model = MyModel(IMAGE_SHAPE)
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='SGD')

### Проверяем работу модели, переобучив на малой выборке

In [ ]:
# transform y indexes to sparse representation: [2] -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
X_dev = X_train[:200]
y_dev = y_train[:200]

y_dev_sparse = tf.keras.utils.to_categorical(y_dev, num_classes=10)

# Check initial loss and overfitting model with small dev dataset
hist = model.fit(X_dev, y_dev_sparse, batch_size=32, epochs=10)

### Обучаем на train, проверяем на val выборке

In [ ]:
BS = 64
LR = 1e-4
EPOCHS = 6

model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='SGD', metrics=['accuracy'])

y_train_sparse = tf.keras.utils.to_categorical(y_train, num_classes=10)
hist = model.fit(X_train, y_train_sparse, validation_split=0.2, batch_size=BS, epochs=EPOCHS)

In [ ]:
plt.figure(figsize=(15, 7))
plt.subplot(221)
plt.title("Train Loss")
plt.plot(hist.history['loss'])
plt.subplot(222)
plt.title("Train/validation accuracy")
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])

# Как видно из графиков, модель переобучилась  

### Задание 3
Поэкспериментируйте с гиперпараметрами и архитектурой, чтобы получить точность на тесте больше 80%

In [ ]:
# Experiment here!


In [ ]:
def multiclass_accuracy(y_pred, y_true):
  return np.mean(np.equal(y_true, np.argmax(y_pred, axis=1)))

y_test_pred = lenet.predict(X_test)
test_accuracy = multiclass_accuracy(test_pred, y_test)

print('Test set accuracy: %f' % (test_accuracy, ))
assert test_accuracy > 0.8